In [1]:
!pip install -q /kaggle/input/kerasv3-lib-ds/tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-deps
!pip install -q /kaggle/input/kerasv3-lib-ds/keras-3.0.4-py3-none-any.whl --no-deps

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/kerasv3-lib-ds/tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/kerasv3-lib-ds/keras-3.0.4-py3-none-any.whl'



In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax" # # you can also use tensorflow or torch

import keras
import keras_nlp
from keras import ops
import tensorflow as tf

import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

import plotly.graph_objs as go
import plotly.express as px

2024-04-16 19:55:46.084431: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 19:55:46.084567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 19:55:46.233058: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
class CFG:
    seed = 42
    preset = "deberta_v3_small_en"
    train_seq_len = 1024
    train_batch_size = 16
    infer_seq_len = 2000
    infer_batch_size = 4
    epochs = 6
    lr_mode = "exp"
    
    labels = [
        "B-EMAIL", "B-ID_NUM", "B-NAME_STUDENT", "B-PHONE_NUM",
        "B-STREET_ADDRESS", "B-URL_PERSONAL", "B-USERNAME",
        "I-ID_NUM", "I-NAME_STUDENT", "I-PHONE_NUM",
        "I-STREET_ADDRESS", "I-URL_PERSONAL", "O"
    ]
    id2label = {index: label for index, label in enumerate(labels)}
    label2id = {label: index for index, label in enumerate(labels)}
    num_labels = len(labels)
    
    train = False

In [4]:
devices = keras.distribution.list_devices()
print("Device:", devices)

if len(devices) > 1:
    data_parallel = keras.distribution.DataParallel(devices=devices)
    keras.distribution.set_distribution(data_parallel)

Device: ['gpu:0', 'gpu:1']


In [5]:
BASE_PATH = "/kaggle/input/pii-detection-removal-from-educational-data"

In [6]:
data = json.load(open(f"{BASE_PATH}/train.json"))

words = np.empty(len(data), dtype=object)
labels = np.empty(len(data), dtype=object)

for i, x in tqdm(enumerate(data), total=len(data)):
    words[i] = np.array(x["tokens"])
    labels[i] = np.array([CFG.label2id[label] for label in x["labels"]])

  0%|          | 0/6807 [00:00<?, ?it/s]

In [7]:
all_labels = np.array([x for label in labels for x in label])
unique_labels, label_counts = np.unique(all_labels, return_counts=True)

In [8]:
train_words, valid_words, train_labels, valid_labels = train_test_split(
    words, labels, test_size=0.2, random_state=CFG.seed
)

tokenizer = keras_nlp.models.DebertaV3Tokenizer.from_preset(
    CFG.preset,
)

packer = keras_nlp.layers.MultiSegmentPacker(
    start_value=tokenizer.cls_token_id,
    end_value=tokenizer.sep_token_id,
    sequence_length=10,
)

sample_words = words[0][:5]
sample_tokens_int = [
    token.tolist() for word in sample_words for token in tokenizer(word)
]
sample_tokens_str = [tokenizer.id_to_token(token) for token in sample_tokens_int]

padded_sample_tokens_int = packer(np.array(sample_tokens_int))[0].tolist()
padded_sample_tokens_str = [
    tokenizer.id_to_token(token) for token in padded_sample_tokens_int
]

Attaching 'tokenizer.json' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...


In [9]:
def get_tokens(words, seq_len, packer):
    token_words = tf.expand_dims(tokenizer(words), axis=-1)
    tokens = tf.reshape(token_words, [-1])
    tokens = packer(tokens)[0][:seq_len]
    inputs = {"token_ids": tokens, "padding_mask": tokens != 0}
    return inputs, tokens, token_words

def get_token_ids(token_words):
    word_ids = tf.range(tf.shape(token_words)[0])
    word_size = tf.reshape(tf.map_fn(lambda word: tf.shape(word)[0:1], token_words), [-1])
    token_ids = tf.repeat(word_ids, word_size)
    return token_ids

def get_token_labels(word_labels, token_ids, seq_len):
    token_labels = tf.gather(word_labels, token_ids)
    mask = tf.concat([[True], token_ids[1:] != token_ids[:-1]], axis=0)
    token_labels = tf.where(mask, token_labels, -100)
    token_labels = token_labels[:seq_len - 2]
    pad_start = 1
    pad_end = seq_len - tf.shape(token_labels)[0] - 1
    token_labels = tf.pad(token_labels, [[pad_start, pad_end]], constant_values=-100)
    return token_labels

def process_token_ids(token_ids, seq_len):
    token_ids = token_ids[:seq_len - 2]
    pad_start = 1
    pad_end = seq_len - tf.shape(token_ids)[0] - 1
    token_ids = tf.pad(token_ids, [[pad_start, pad_end]], constant_values=-1)
    return token_ids

def process_data(seq_len=720, has_label=True, return_ids=False):
    packer = keras_nlp.layers.MultiSegmentPacker(
        start_value=tokenizer.cls_token_id,
        end_value=tokenizer.sep_token_id,
        sequence_length=seq_len,
    )

    def process(x):
        inputs, tokens, words_int = get_tokens(x["words"], seq_len, packer)
        token_ids = get_token_ids(words_int)
        if has_label:
            token_labels = get_token_labels(x["labels"], token_ids, seq_len)
            return inputs, token_labels
        elif return_ids:
            token_ids = process_token_ids(token_ids, seq_len)
            return token_ids
        else:
            return inputs

    return process


def build_dataset(words, labels=None, return_ids=False, batch_size=4,
                  seq_len=512, shuffle=False, cache=True, drop_remainder=True):
    AUTO = tf.data.AUTOTUNE 

    slices = {"words": tf.ragged.constant(words)}
    if labels is not None:
        slices.update({"labels": tf.ragged.constant(labels)})

    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.map(process_data(seq_len=seq_len,
                             has_label=labels is not None, 
                             return_ids=return_ids), num_parallel_calls=AUTO) 
    ds = ds.cache() if cache else ds  
    if shuffle: 
        ds = ds.shuffle(1024, seed=CFG.seed)  
        opt = tf.data.Options() 
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)  
    ds = ds.prefetch(AUTO) 
    return ds


In [10]:
train_ds = build_dataset(train_words, train_labels,  batch_size=CFG.train_batch_size,
                         seq_len=CFG.train_seq_len, shuffle=True)

valid_ds = build_dataset(valid_words, valid_labels, batch_size=CFG.train_batch_size, 
                         seq_len=CFG.train_seq_len, shuffle=False)

In [11]:
class CrossEntropy(keras.losses.SparseCategoricalCrossentropy):
    def __init__(self, ignore_class=-100, reduction=None, **args):
        super().__init__(reduction=reduction, **args)
        self.ignore_class = ignore_class

    def call(self, y_true, y_pred):
        y_true = ops.reshape(y_true, [-1])
        y_pred = ops.reshape(y_pred, [-1, CFG.num_labels])
        loss = super().call(y_true, y_pred)
        if self.ignore_class is not None:
            valid_mask = ops.not_equal(
                y_true, ops.cast(self.ignore_class, y_pred.dtype)
            )
            loss = ops.where(valid_mask, loss, 0.0)
            loss = ops.sum(loss)
            loss /= ops.maximum(ops.sum(ops.cast(valid_mask, loss.dtype)), 1)
        else:
            loss = ops.mean(loss)
        return loss

In [12]:
class FBetaScore(keras.metrics.FBetaScore):
    def __init__(self, ignore_classes=[-100, 12], average="micro", beta=5.0,
                 name="f5_score", **args):
        super().__init__(beta=beta, average=average, name=name, **args)
        self.ignore_classes = ignore_classes or []

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = ops.convert_to_tensor(y_true, dtype=self.dtype)
        y_pred = ops.convert_to_tensor(y_pred, dtype=self.dtype)
        
        y_true = ops.reshape(y_true, [-1])
        y_pred = ops.reshape(y_pred, [-1, CFG.num_labels])
            
        valid_mask = ops.ones_like(y_true, dtype=self.dtype)
        if self.ignore_classes:
            for ignore_class in self.ignore_classes:
                valid_mask &= ops.not_equal(y_true, ops.cast(ignore_class, y_pred.dtype))
        valid_mask = ops.expand_dims(valid_mask, axis=-1)
        
        y_true = ops.one_hot(y_true, CFG.num_labels)
        
        if not self._built:
            self._build(y_true.shape, y_pred.shape)
        
        threshold = ops.max(y_pred, axis=-1, keepdims=True)
        y_pred = ops.logical_and(
            y_pred >= threshold, ops.abs(y_pred) > 1e-9
        )

        y_pred = ops.cast(y_pred, dtype=self.dtype)
        y_true = ops.cast(y_true, dtype=self.dtype)
        
        tp = ops.sum(y_pred * y_true * valid_mask, self.axis)
        fp = ops.sum(y_pred * (1 - y_true) * valid_mask, self.axis)
        fn = ops.sum((1 - y_pred) * y_true * valid_mask, self.axis)
        
        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

In [13]:
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
    CFG.preset,
)
backbone_output = backbone.output
logits = keras.layers.Dense(CFG.num_labels, name="logits")(backbone_output)
predictions = keras.layers.Activation("softmax", dtype="float32", name="prediction")(logits)
model = keras.Model(inputs=backbone.input, outputs=predictions)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=2e-5),
    loss=CrossEntropy(),
    metrics=[FBetaScore()]
)

Attaching 'config.json' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...


In [14]:
if CFG.train:
    history = model.fit(
        train_ds,  
        validation_data=valid_ds, 
        epochs=CFG.training_epochs,  
        callbacks=[lr_cb],  
        verbose=1 
    )
else:
    model.load_weights("/kaggle/input/model-weights-h5/model.weights.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 214 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [15]:
valid_ds = build_dataset(valid_words, valid_labels, return_ids=False, batch_size=CFG.infer_batch_size, seq_len=CFG.infer_seq_len, shuffle=False, cache=False)

model.evaluate(valid_ds, return_dict=True, verbose=0)
test_data = json.load(open(f"{BASE_PATH}/test.json"))

need_samples  = len(devices) - len(test_data) % len(devices)
for _ in range(need_samples):
    test_data.append(test_data[-1])
    
test_words = np.empty(len(test_data), dtype=object)
test_docs = np.empty(len(test_data), dtype=np.int32)

for i, x in tqdm(enumerate(test_data), total=len(test_data)):
    test_words[i] = np.array(x["tokens"])
    test_docs[i] = x["document"]

id_ds = build_dataset(test_words, return_ids=True, batch_size=len(test_words), 
                      seq_len=CFG.infer_seq_len, shuffle=False, cache=False, drop_remainder=False)
test_token_ids = ops.convert_to_numpy([ids for ids in iter(id_ds)][0])

test_ds = build_dataset(test_words, return_ids=False, batch_size=CFG.infer_batch_size,
                        seq_len=CFG.infer_seq_len, shuffle=False, cache=False, drop_remainder=False)

test_preds = model.predict(test_ds, verbose=1)

test_preds = np.argmax(test_preds, axis=-1)

test_docs = test_docs[:-need_samples]
test_token_ids = test_token_ids[:-need_samples]
test_preds = test_preds[:-need_samples]
test_words = test_words[:-need_samples]

document_list = []
token_id_list = []
label_id_list = []
token_list = []

for doc, token_ids, preds, tokens in tqdm(
    zip(test_docs, test_token_ids, test_preds, test_words), total=len(test_words)
):
    mask1 = np.concatenate(([True], token_ids[1:] != token_ids[:-1]))
    mask2 = (preds != 12)
    mask3 = (token_ids != -1)
    mask = (mask1 & mask2 & mask3)
    
    token_ids = token_ids[mask]
    preds = preds[mask]

    if len(token_ids):
        token_list.extend(tokens[token_ids])
        document_list.extend([doc] * len(token_ids))
        token_id_list.extend(token_ids)
        label_id_list.extend(preds)

pred_df = pd.DataFrame(
    {
        "document": document_list,
        "token": token_id_list,
        "label_id": label_id_list,
        "token_string": token_list,
    }
)
pred_df = pred_df.rename_axis("row_id").reset_index()
pred_df["label"] = pred_df.label_id.map(CFG.id2label)
pred_df.head(10)

sub_df = pred_df.drop(columns=["token_string", "label_id"])
sub_df.to_csv("submission.csv", index=False)


  0%|          | 0/12 [00:00<?, ?it/s]

3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step


  0%|          | 0/10 [00:00<?, ?it/s]